# engram Memory System — Research Notebook

This notebook documents the evaluation methodology, results, and improvement trajectory
for the engram AI memory system.

## Key metrics
- **Token-F1**: Lexical overlap between predicted and gold answers
- **LLM-judge**: Semantic correctness scored by gemini-2.5-flash (YES/NO)
- **Not-found rate**: % questions where engram returned nothing
- **Ceiling efficiency**: % of full-context (28K token) baseline we achieve

## Eval datasets
1. **Domain eval** (`eval/qa_dataset.json`): 385 QA pairs generated from engram's own knowledge.
   Tests the full pipeline on its actual domain (Claude Code sessions).
2. **LoCoMo-10** (`eval/locomo10.json`): Official long-term conversational memory benchmark.
   10 multi-session conversations, 1986 QA pairs across 5 categories.

In [ ]:
import json
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

ROOT = Path('..')  # repo root from eval/
EVAL_DIR = Path('.')

def load(path):
    p = EVAL_DIR / path
    if p.exists():
        return json.loads(p.read_text())
    return {}

print('Notebook ready')

## 1. Domain Eval Results — Improvement Trajectory

In [ ]:
# Load all domain eval results
runs = [
    ('v1 baseline\n(raw add, t=0.4)', load('domain_results_v1.json')),
    ('v2 improvements\n(HyDE, t=0.15)', load('domain_results_v3.json')),
    ('v3 full 385Q\n(HyDE+index, t=0.2)', load('domain_results_v3_full.json')),
    ('Ceiling\n(full context)', load('domain_results_ceiling.json')),
]

labels = [r[0] for r in runs if r[1]]
f1_scores = [r[1].get('overall_f1', 0) for r in runs if r[1]]
judge_scores = [r[1].get('overall_judge', 0) for r in runs if r[1]]
not_found = [r[1].get('not_found_rate', 0) for r in runs if r[1]]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('engram Domain Eval — Improvement Trajectory', fontsize=14, fontweight='bold')

colors = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']
x = np.arange(len(labels))

# F1
axes[0].bar(x, f1_scores, color=colors[:len(labels)], alpha=0.85, edgecolor='white')
axes[0].axhline(67.9, color='green', linestyle='--', alpha=0.5, label='ceiling')
axes[0].set_title('Token-F1')
axes[0].set_xticks(x); axes[0].set_xticklabels(labels, fontsize=8)
axes[0].set_ylim(0, 80)
for i, v in enumerate(f1_scores): axes[0].text(i, v+1, f'{v:.1f}', ha='center', fontsize=9, fontweight='bold')
axes[0].legend()

# Judge
axes[1].bar(x, judge_scores, color=colors[:len(labels)], alpha=0.85, edgecolor='white')
axes[1].axhline(14.0, color='green', linestyle='--', alpha=0.5, label='ceiling')
axes[1].axhline(67.13, color='orange', linestyle='--', alpha=0.5, label='Mem0 SOTA')
axes[1].set_title('LLM-Judge Score')
axes[1].set_xticks(x); axes[1].set_xticklabels(labels, fontsize=8)
axes[1].set_ylim(0, 80)
for i, v in enumerate(judge_scores): axes[1].text(i, v+0.3, f'{v:.1f}', ha='center', fontsize=9, fontweight='bold')
axes[1].legend()

# Not-found
axes[2].bar(x, not_found, color=colors[:len(labels)], alpha=0.85, edgecolor='white')
axes[2].axhline(6.5, color='green', linestyle='--', alpha=0.5, label='ceiling (6.5%)')
axes[2].set_title('Not-Found Rate %')
axes[2].set_xticks(x); axes[2].set_xticklabels(labels, fontsize=8)
for i, v in enumerate(not_found): axes[2].text(i, v+0.5, f'{v:.1f}%', ha='center', fontsize=9, fontweight='bold')
axes[2].legend()

plt.tight_layout()
plt.savefig('domain_eval_trajectory.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'F1 improvement:    {f1_scores[0]:.1f} → {f1_scores[-2]:.1f}  (+{f1_scores[-2]-f1_scores[0]:.1f})')
print(f'Judge improvement: {judge_scores[0]:.1f} → {judge_scores[-2]:.1f}  (+{judge_scores[-2]-judge_scores[0]:.1f})')
print(f'Not-found:         {not_found[0]:.1f}% → {not_found[-2]:.1f}%  (-{not_found[0]-not_found[-2]:.1f}pp)')

## 2. Per-Category Breakdown

In [ ]:
categories = ['decisions', 'solutions', 'patterns', 'bugs', 'insights', 'procedures']

baseline = load('domain_results_v1.json').get('by_category', {})
current  = load('domain_results_v3_full.json').get('by_category', {})
ceiling  = load('domain_results_ceiling.json').get('by_category', {})

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Per-Category: Baseline vs Current vs Ceiling', fontsize=13, fontweight='bold')

x = np.arange(len(categories))
w = 0.28

for ax, metric, title in [
    (axes[0], 'f1', 'Token-F1 by Category'),
    (axes[1], 'judge', 'LLM-Judge by Category'),
]:
    v_base = [baseline.get(c, {}).get(metric, 0) for c in categories]
    v_curr = [current.get(c, {}).get(metric, 0) for c in categories]
    v_ceil = [ceiling.get(c, {}).get(metric, 0) for c in categories]

    ax.bar(x - w, v_base, w, label='Baseline', color='#e74c3c', alpha=0.8)
    ax.bar(x,     v_curr, w, label='Current',  color='#3498db', alpha=0.8)
    ax.bar(x + w, v_ceil, w, label='Ceiling',  color='#2ecc71', alpha=0.8)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, rotation=20, ha='right', fontsize=9)
    ax.legend(fontsize=8)
    ax.set_ylim(0, 75)

plt.tight_layout()
plt.savefig('category_breakdown.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. LoCoMo Benchmark Results

In [ ]:
# LoCoMo results collected from background runs
locomo_versions = {
    'v1\n(raw add)': {'f1': 18.0, 'judge': None, 'by_cat': {
        'single_hop': 19.1, 'temporal': 5.2, 'open_ended': 6.0,
        'multi_hop': 23.9, 'multi_hop_cs': 9.1
    }},
    'v2\n(fact+concise)': {'f1': 35.6, 'judge': None, 'by_cat': {
        'single_hop': 30.5, 'temporal': 17.3, 'open_ended': 3.2,
        'multi_hop': 48.1, 'multi_hop_cs': 0.0
    }},
    'v3\n(2.5-pro+atomic)': {'f1': 45.4, 'judge': 49.5, 'by_cat': {
        'single_hop': None, 'temporal': None, 'open_ended': None,
        'multi_hop': None, 'multi_hop_cs': None
    }},
}

# Baselines from literature
baselines = {
    'GPT-4\n(no memory)': {'f1': 32.1, 'judge': None},
    'Mem0\n(token F1)':   {'f1': 38.72, 'judge': None},
    'Mem0\n(LLM judge)':  {'f1': None, 'judge': 67.13},
    'Human\nceiling':     {'f1': 87.9, 'judge': None},
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('LoCoMo Benchmark — engram vs Baselines', fontsize=13, fontweight='bold')

# F1 comparison
ax = axes[0]
all_f1 = {**{k: v['f1'] for k, v in locomo_versions.items() if v['f1']},
          **{k: v['f1'] for k, v in baselines.items() if v['f1']}}
sorted_f1 = sorted(all_f1.items(), key=lambda x: x[1])
names = [x[0] for x in sorted_f1]
vals = [x[1] for x in sorted_f1]
colors_f1 = ['#3498db' if 'engram' not in n and 'v' not in n.lower() else '#e74c3c' for n in names]
engram_colors = {'v1': '#e74c3c', 'v2': '#f39c12', 'v3': '#9b59b6'}
bar_colors = []
for n in names:
    if 'v1' in n: bar_colors.append('#e74c3c')
    elif 'v2' in n: bar_colors.append('#f39c12')
    elif 'v3' in n: bar_colors.append('#9b59b6')
    else: bar_colors.append('#95a5a6')

bars = ax.barh(names, vals, color=bar_colors, alpha=0.85, edgecolor='white')
ax.axvline(38.72, color='orange', linestyle='--', alpha=0.7, label='Mem0 token-F1')
ax.set_title('Token-F1 (higher = better)')
ax.set_xlabel('F1 Score')
for bar, val in zip(bars, vals):
    ax.text(val + 0.5, bar.get_y() + bar.get_height()/2, f'{val:.1f}',
            va='center', fontsize=9, fontweight='bold')
ax.legend()

# Judge comparison  
ax2 = axes[1]
all_judge = {k: v['judge'] for k, v in locomo_versions.items() if v.get('judge')}
all_judge['Mem0\n(LLM judge)'] = 67.13
sorted_j = sorted(all_judge.items(), key=lambda x: x[1])
jnames = [x[0] for x in sorted_j]
jvals = [x[1] for x in sorted_j]
jcolors = ['#9b59b6' if 'v3' in n else '#95a5a6' for n in jnames]
jbars = ax2.barh(jnames, jvals, color=jcolors, alpha=0.85, edgecolor='white')
ax2.axvline(67.13, color='orange', linestyle='--', alpha=0.7, label='Mem0 LLM-judge')
ax2.set_title('LLM-as-Judge (higher = better)')
ax2.set_xlabel('Judge Score %')
for bar, val in zip(jbars, jvals):
    ax2.text(val + 0.5, bar.get_y() + bar.get_height()/2, f'{val:.1f}',
             va='center', fontsize=9, fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.savefig('locomo_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nKey comparison:')
print(f'engram v3 token-F1:  ~45.4  vs  Mem0: 38.72  → +6.7 BEATS Mem0 token-F1')
print(f'engram v3 LLM-judge: ~49.5  vs  Mem0: 67.13  → -17.6 gap to Mem0 headline')

## 4. Retrieval Gap Analysis

In [ ]:
# Ceiling efficiency analysis
ceiling_f1    = 67.9
ceiling_judge = 14.0

iterations = [
    {'label': 'v1 baseline\nt=0.4 k=5\nno HyDE', 'f1': 38.9, 'judge': 8.8, 'not_found': 31.2},
    {'label': 'v3 (90Q)\nt=0.15 k=12\n+HyDE+index', 'f1': 53.7, 'judge': 15.0, 'not_found': 15.0},
    {'label': 'v3 full (385Q)\nt=0.2 k=8\n+HyDE+index', 'f1': 56.6, 'judge': 15.3, 'not_found': 18.4},
    {'label': 'Ceiling\nfull 28K ctx\nno retrieval', 'f1': ceiling_f1, 'judge': ceiling_judge, 'not_found': 6.5},
]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Retrieval Efficiency vs Ceiling', fontsize=13, fontweight='bold')

x = np.arange(len(iterations))
colors_iter = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']

for ax, metric, title, ceil_val in [
    (axes[0], 'f1', 'Token-F1', ceiling_f1),
    (axes[1], 'judge', 'LLM-Judge', ceiling_judge),
]:
    vals = [it[metric] for it in iterations]
    bars = ax.bar(x, vals, color=colors_iter, alpha=0.85, edgecolor='white')
    ax.axhline(ceil_val, color='green', linestyle='--', lw=2, label=f'ceiling ({ceil_val})')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels([it['label'] for it in iterations], fontsize=7.5)
    ax.legend()
    for bar, val in zip(bars, vals):
        pct = val / ceil_val * 100
        ax.text(bar.get_x() + bar.get_width()/2, val + 0.2,
                f'{val:.1f}\n({pct:.0f}%)', ha='center', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.savefig('ceiling_efficiency.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nEfficiency summary:')
for it in iterations:
    eff_f1 = it['f1'] / ceiling_f1 * 100
    eff_j  = it['judge'] / ceiling_judge * 100
    print(f'  {it["label"].split(chr(10))[0]:<20} F1={eff_f1:.0f}%  judge={eff_j:.0f}%  not_found={it["not_found"]:.1f}%')

## 5. Feature Impact Analysis

Which improvements contributed how much?

In [ ]:
features = [
    ('Update Resolver\n(ADD/UPDATE/NOOP)', 'Reduces knowledge duplication,\nimproves answer precision', 'indirect'),
    ('FadeMem Decay\n(strength field)', 'Old knowledge decays,\nfresh knowledge surfaces first', 'indirect'),
    ('HyDE for ask\n(hypothetical doc embed)', 'Better semantic match\nespecially for implicit queries', '+8-12 F1'),
    ('Concise mode\n(--concise flag)', 'Short answers = better\ntoken-F1 match', '+18 F1'),
    ('Fresh embedding index\n(procedures 3→21 chunks)', 'Previously uncovered\ncategory now retrievable', '+39 F1 procedures'),
    ('threshold 0.4→0.15\ntop-k 5→12', 'Wider retrieval net\nnot-found 31%→18%', '+8 F1'),
    ('Fact extraction\n(2.5-pro + entity tags)', 'LoCoMo: atomic facts with\n[Person:X][Date:Y] anchors', '+16 F1 LoCoMo'),
]

fig, ax = plt.subplots(figsize=(14, 6))
fig.suptitle('Feature Impact Map', fontsize=13, fontweight='bold')

# Rough impact scores for visualization
impacts = [3, 3, 8, 18, 15, 8, 16]
feature_labels = [f[0] for f in features]
colors_feat = ['#95a5a6', '#95a5a6', '#3498db', '#e74c3c', '#f39c12', '#2ecc71', '#9b59b6']

bars = ax.barh(feature_labels, impacts, color=colors_feat, alpha=0.85, edgecolor='white')
ax.set_xlabel('Approximate F1 impact')
ax.set_title('Estimated F1 contribution per feature')
for bar, (feat, desc, impact) in zip(bars, features):
    ax.text(bar.get_width() + 0.2, bar.get_y() + bar.get_height()/2,
            impact, va='center', fontsize=8, color='gray')

gray_patch = mpatches.Patch(color='#95a5a6', label='Architecture (indirect)')
blue_patch  = mpatches.Patch(color='#3498db', label='Retrieval quality')
red_patch   = mpatches.Patch(color='#e74c3c', label='Synthesis quality')
orange_patch = mpatches.Patch(color='#f39c12', label='Index coverage')
green_patch  = mpatches.Patch(color='#2ecc71', label='Threshold tuning')
purple_patch = mpatches.Patch(color='#9b59b6', label='Domain extraction')
ax.legend(handles=[gray_patch, blue_patch, red_patch, orange_patch, green_patch, purple_patch],
          loc='lower right', fontsize=8)

plt.tight_layout()
plt.savefig('feature_impact.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. How to Run Evals

```bash
# One-time setup: generate QA dataset from real knowledge
python eval/gen_qa_dataset.py --project Personal --output eval/qa_dataset.json

# Run eval (saves timestamped results, CI-gated)
bash eval/run_eval.sh

# Compare all runs
python eval/compare.py eval/results/*.json

# Automated improvement loop
python eval/improvement_loop.py --project Personal --target-judge 13.5 --max-per-cat 20

# Full ceiling baseline (no retrieval, full context)
python eval/engram_eval.py --project Personal --dataset eval/qa_dataset.json \
    --full-context --use-judge --output eval/domain_results_ceiling.json

# LoCoMo benchmark
python eval/locomo_eval.py --engram ./target/release/engram --use-judge --workers 3
```

## 7. Gap Analysis & Next Steps

In [ ]:
print('''
╔══════════════════════════════════════════════════════════════╗
  Current State (Feb 2026)
╠══════════════════════════════════════════════════════════════╣

  Domain eval (385 QA pairs, engram's actual domain):
    Token-F1:   56.6  (83% of ceiling)
    LLM-judge:  15.3  (109% of ceiling — concise beats full-context!)
    Not found:  18.4% (ceiling: 6.5%)

  LoCoMo-10 (social conversations):
    v3 token-F1:  ~45.4  (BEATS Mem0 token-F1 of 38.72)
    v3 LLM-judge: ~49.5  (gap to Mem0 67.13: -17.6)

╠══════════════════════════════════════════════════════════════╣
  Remaining gaps:

  1. Not-found 18.4% → 6.5% (ceiling)
     Fix: better chunking, adaptive threshold per category

  2. procedures judge 4.0% (procedural lists score poorly)
     Fix: multi-item answer format, step-by-step eval metric

  3. LoCoMo judge 49.5% vs Mem0 67.13%
     Fix: entity graph retrieval, temporal-aware indexing

  4. decisions judge 11.1% (complex multi-part decisions)
     Fix: decompose decisions into atomic sub-facts

╠══════════════════════════════════════════════════════════════╣
  Run improvement_loop.py to automate the fix cycle.
╚══════════════════════════════════════════════════════════════╝
''')